<a href="https://colab.research.google.com/github/SquadronHawk/CocktailRecipeNN/blob/main/LSTM_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.layers import LSTM, Dense, Input, Bidirectional, Dropout
import numpy as np
import os
import time
import io
import random

In [ ]:
## This was done in google colab, as I had issues getting Tensorflow working on my machine. The following code may not be 100% compatible with the repo, but it should be easy to fix.
from google.colab import files
uploaded = files.upload()

Saving with_names.txt to with_names (1).txt


In [ ]:
file = open('with_names.txt')

In [ ]:
text = file.read()


In [ ]:
print(len(text))

23578


In [ ]:
chars = sorted(list(set(text)))
print(len(chars))

71


In [ ]:
# Here we are hand-encoding the letters in our file.

char_index = {u:i for i, u in enumerate(chars)}
index_char = {i:u for i, u in enumerate(chars)}

In [ ]:
#Creating our lists for what the text is trying to predict. I will continue playing with these values.

max_len = 40
step = 3
recipes = []
next_chars = []
for i in range(0, len(text)- max_len, step):
  recipes.append(text[i : i + max_len])
  next_chars.append(text[i + max_len])

In [ ]:
# Making our boolean arrays to feed into the NN

x = np.zeros((len(recipes), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(recipes), len(chars)), dtype=np.bool)
for i, recipe in enumerate(recipes):
    for t, char in enumerate(recipe):
        x[i, t, char_index[char]] = 1
    y[i, char_index[next_chars[i]]] = 1

In [ ]:
# After attempting a few different models, this is our current best. I will continue to try to make a more consistent model.

model = keras.Sequential(
    [
        keras.Input(shape=(max_len, len(chars))),
        keras.layers.LSTM(128),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(len(chars), activation="softmax"),
    ]
)
    

optimizer = keras.optimizers.RMSprop(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", )

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               102400    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 71)                9159      
Total params: 111,559
Trainable params: 111,559
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
epochs = 40
batch_size = 128

## To see our model train in real-time, this will print the outputs for each epoch in a variety of diversities of our choosing.

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.4, 0.75, 1, 1.25, 2]:
        print("...Diversity:", diversity)

        generated = ""
        recipe = text[start_index : start_index + max_len]
        print('...Generating with seed: "' + recipe + '"')

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(recipe):
                x_pred[0, t, char_index[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_char[next_index]
            recipe = recipe[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

62/62 [==============================] - 7s 119ms/step - loss: 7.1739

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "1 oz Green Chartreuse, 1 oz Maraschino L"
...Generated:  iliilililililiililillilillilllillillillillllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll

...Diversity: 0.4
...Generating with seed: "1 oz Green Chartreuse, 1 oz Maraschino L"
...Generated:  iiilililililiillilliii.lilllllirilil.iiallllllllllllllllllllllilllllllil.lllilAllllllllllllllllllllrllllllllllllllllllllllrlllllllllrllllllllllllllllllllllllllllllllllllAlllllllllllllrlllllllllllllllllllllllllllAllllllll,llAllAlllllllllllllllllllllllllrlllllllllllllllllllllllllllllllllllllllll

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...Generated:  5 oz Gin, 0.5 oz Sparkitthrnonapple Syrup, 0.75 oz Lime Juice, 0.75 oz Lime Juice, 0.5 oz Gin, 0.75 oz Lime Juice, 0.5 oz Gin, 0.75 oz Lime Juice, 0.5 oz Lime Juice, 0.5 oz Gin, 0.75 oz Gin, 0.75 oz Gin, 0.75 oz Gin, 0.75 oz Gin, 0.75 oz Lime Juice, 0.5 oz Gin, 0.5 oz Lime Juice, 0.5 oz Gin, 0.75 oz Lin, 0.75 oz Gin, 0.75 oz Gin, 0.75 oz Gin, 0.5 oz Lime Juice, 0.75 oz Gin, 0.75 oz Lime Juice, 0.5

...Diversity: 0.4
...Generating with seed: "e Juice
San Francisco: 1.5 oz Vodka, 0.7"
...Generated:  5 oz Gin, 0.5 oz Gin, 0.5 oz Gin, 0.75 oz Soda Water
Apberry Lingerry Liqueur, 0.5 oz Gin, 0.5 oz Lime Juice, 0.75 oz Gin, 0.75 oz Simple Syrup, 0.75 oz Gin, 0.5 oz Gin, 0.5 oz Gin, 0.5 oz Lime Juice, 1 oz Lime Juice, 0.75 oz Simple Syrup
Rodaranberry Liqueur, 0.75 oz Lime Juice, 0.75 oz Gin, 0.75 oz Lime Juice
Blackberry Liqueur, 0.75 oz Aplino, 0.5 oz Gin, 0.5 oz Gin, 1 oz Gin, 0.75 oz Gin, 0.5 

...Diversity: 0.75
...Generating with seed: "e Juice
San Francisco: 1.5 oz Vodka

In [ ]:
## Our model is able to make fun cocktails but they are not consistently drinkable.
## Using the no_names.txt file, this allows us to get more consistently drinkable recipes
## (Albeit with less fun results)

## My next step is to train a model that will generate names based on recipes loaded into it

## I'm far from finished with this capstone, I know. But I will continue working on this now that the class is over.